<h1> Random Forest <h1>

<h2> Mapa </h2>
<ul>
    <li>1-. Formular la pregunta y determinar los datos requeridos</li>
    <li>2-. Adquirir los datos en un formato accesible</li>
    <li>3-. Identifique y corrija los puntos de datos faltantes/anomalías según sea necesario</li>
    <li>4-. Preparar los datos para el modelo de aprendizaje automático</li>
    <li>6-. Entrenar el modelo en los datos de entrenamiento</li>
    <li>7-. Hacer predicciones sobre los datos de prueba</li>
    <li>8-. Compare las predicciones con los objetivos conocidos del conjunto de pruebas y calcule las métricas de rendimiento</li>
    <li>9-. Si el rendimiento no es satisfactorio, ajuste el modelo, adquiera más datos o pruebe una técnica de modelado diferente</li>
    <li>10-. Interprete el modelo y reporte los resultados visual y numéricamente</li>
</ul>

# 2-. Adquisición de datos

Primero, necesitamos algunos datos. Para usar un ejemplo realista, recuperé datos de pacientes del Hospital Herminda Martin de Chillan. En general, aproximadamente el 80 % del tiempo dedicado al análisis de datos es limpiar y recuperar datos, pero esta carga de trabajo se puede reducir encontrando fuentes de datos de alta calidad.

In [ ]:
# Libreria para la manipulación de los datos
import pandas as pd

# Leer el dataframe
dataframe = pd.read_excel('../bdd/dataframe.xlsx')
dataframe.head(5)

In [ ]:
print('Existen {} pacientes con {} variables.'.format(*dataframe.shape))

# 3-. Identifique y corrija los puntos de datos faltantes/anomalías según sea necesario

In [ ]:
# Tomaremos las variables más significativas para la investigación
columnasMuestra = ['HTA', 'DIABETES', 'EDAD','TRIGLICERIDOS', 'PESO', 'TALLA', 'FC', 'GLUCOSA', 'Hb A/C  %', 'COL. TOTAL', 'TRIGLICERIDOS', 'LDL', 'HDL', 'HCTO', 'HB', 'VCM', 'HCM', 'VHS', 'PLAQUETAS', 'INR', 'CONTEO G.B.', 'P.C.R', 'Nitrogeno Ureico', 'Uremia', 'Creatinina', 'TTPA', 'TP', 'NA', 'K', 'CL', 'Fosfatasa Alcalina', 'Gamma glutamil', 'Transaminasa piruvica', 'Trans oxal', 'GLASGOW AL INICO ACV', 'NIHSS_INICO_ACV', 'RANKIN INICIO ACV', 'NIHSS alta ACV', 'RANKIN alta ACV', 'NIHSS 6M', 'RANKIN 6M']
dataset = dataframe[[*columnasMuestra]]

# Para los datos fallecidos y NaN realizamos la impuntación y manejo de datos a través del promedio
dataset['NIHSS 6M'] = dataset['NIHSS 6M'].replace( ['fallecido'], 0)
dataset['RANKIN 6M'] = dataset['RANKIN 6M'].replace( ['no'], 0)

valores_por_defecto = {'HTA': "DESCONOCIDO",
                       'DIABETES' : "DESCONOCIDO",
                       'EDAD':dataset["EDAD"].mean(),
                       'TRIGLICERIDOS':dataset["TRIGLICERIDOS"].mean(), 
                       'PESO':dataset["PESO"].mean(),
                       'TALLA':dataset["TALLA"].mean(),
                       'FC':dataset["FC"].mean(),
                       'GLUCOSA':dataset["GLUCOSA"].mean(),
                       'Hb A/C  %':dataset["Hb A/C  %"].mean(), 
                       'COL. TOTAL':dataset["COL. TOTAL"].mean(), 
                       'TRIGLICERIDOS':dataset["TRIGLICERIDOS"].mean(),
                       'LDL':dataset["LDL"].mean(), 
                       'HDL':dataset["HDL"].mean(),
                       'HCTO':dataset["HCTO"].mean(), 
                       'HB':dataset["HB"].mean(),
                       'VCM':dataset["VCM"].mean(),
                       'HCM':dataset["HCM"].mean(), 
                       'VHS':dataset["VHS"].mean(),
                       'PLAQUETAS':dataset["PLAQUETAS"].mean(),
                       'INR':dataset["INR"].mean(),
                       'CONTEO G.B.':dataset["CONTEO G.B."].mean(),
                       'P.C.R':dataset["P.C.R"].mean(),
                       'Nitrogeno Ureico':dataset["Nitrogeno Ureico"].mean(),
                       'Uremia':dataset["Uremia"].mean(),
                       'Creatinina':dataset["Creatinina"].mean(), 
                       'TTPA':dataset["TTPA"].mean(),
                       'TP':dataset["TP"].mean(), 
                       'NA':dataset["NA"].mean(), 
                       'K':dataset["K"].mean(),
                       'CL':dataset["CL"].mean(),
                       'Fosfatasa Alcalina':dataset["Fosfatasa Alcalina"].mean(),
                       'Gamma glutamil':dataset["Gamma glutamil"].mean(), 
                       'Transaminasa piruvica':dataset["Transaminasa piruvica"].mean(),
                       'Trans oxal':dataset["Trans oxal"].mean(), 
                       'GLASGOW AL INICO ACV':dataset["GLASGOW AL INICO ACV"].mean(),
                       'NIHSS_INICO_ACV':dataset["NIHSS_INICO_ACV"].mean().round(), 
                       'RANKIN INICIO ACV':dataset["RANKIN INICIO ACV"].mean(), 
                       'NIHSS alta ACV':dataset["NIHSS alta ACV"].mean(), 
                       'RANKIN alta ACV':dataset["RANKIN alta ACV"].mean(), 
                       'NIHSS 6M':dataset["NIHSS 6M"].mean(), 
                       'RANKIN 6M':dataset["RANKIN 6M"].mean()
                      }

dataset = dataset.fillna(value=valores_por_defecto)
dataset.head(5)

### Inspección numérica y visual de datos

In [ ]:
round(dataset.describe(), 2)

### Número de clase

In [ ]:
print(dataframe.groupby('NIHSS_INICO_ACV').size())

In [ ]:
# Obtener la columna a comparar
clase = dataset['NIHSS_INICO_ACV']

In [ ]:
# Para gráficos matpltlib
import matplotlib.pyplot as plt
%matplotlib inline

#Establecer estilo
plt.style.use('fivethirtyeight')

In [ ]:
# Configurar el diseño de trazado
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize = (15,10))
fig.autofmt_xdate(rotation = 45)

# GLASGOW
ax1.plot(clase, dataset['GLASGOW AL INICO ACV'])
ax1.set_xlabel(''); ax1.set_ylabel('Escala'); ax1.set_title('GLASGOW AL INICO')

# RANKIN INICIO ACV
ax2.plot(clase, dataset['RANKIN INICIO ACV'])
ax2.set_xlabel(''); ax2.set_ylabel('Escala'); ax2.set_title('RANKIN INICIO ACV')

# NIHSS alta ACV
ax3.plot(clase, dataset['NIHSS alta ACV'])
ax3.set_xlabel('NIHSS_INICO_ACV'); ax3.set_ylabel('Escala'); 
ax3.set_title('NIHSS alta ACV')

# NIHSS 6M
ax4.plot(clase, dataset['NIHSS 6M'])
ax4.set_xlabel('NIHSS_INICO_ACV'); ax4.set_ylabel('Escala'); ax4.set_title('NIHSS 6M')

plt.tight_layout(pad=2)

In [ ]:
# Configurar el diseño de trazado
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize = (15,10))
fig.autofmt_xdate(rotation = 45)

# Glucosa
ax1.plot(clase, dataset['GLUCOSA'])
ax1.set_xlabel(''); ax1.set_ylabel('Cantidad (mg/dl)'); ax1.set_title('GLUCOSA')

# Trigliseridos
ax2.plot(clase, dataset['TRIGLICERIDOS'], 'r-')
ax2.set_xlabel(''); ax2.set_ylabel('Cantidad (mmol/L)'); ax2.set_title('TRIGLICERIDOS')

# NItrogeno
ax3.plot(clase, dataset['Nitrogeno Ureico'])
ax3.set_xlabel('NIHSS_INICO_ACV'); ax3.set_ylabel('Cantidad (mg/dL)'); ax3.set_title('Nitrogeno Ureico')

# Uremia
ax4.plot(clase, dataset['Uremia'], 'ro')
ax4.set_xlabel('NIHSS_INICO_ACV'); ax4.set_ylabel('Cantidad (mg/dL)'); ax4.set_title('Uremia')

plt.tight_layout(pad=2)

### Parcelas

In [ ]:
# Cree columnas de estaciones para emparejar colores de trazado
seasons = []

# Mosatramos solo algunos atributos
reduccion_dataset = dataset[['GLASGOW AL INICO ACV', 'NIHSS_INICO_ACV', 'NIHSS alta ACV', 'NIHSS 6M']]


# Usar seaborn para plots
import seaborn as sns
sns.set(style="ticks", color_codes=True);

# Create a custom color palete
palette = sns.xkcd_palette(['dark blue', 'dark green', 'gold', 'orange'])

# Make the pair plot with a some aesthetic changes
sns.pairplot(reduccion_dataset, diag_kind = 'kde', palette= palette, plot_kws=dict(alpha = 0.7),
                   diag_kws=dict(shade=True)); 

# 4-. Preparar los datos para el modelo de aprendizaje automático
## Preparación de datos

Los pasos exactos para la preparación de los datos dependerán del modelo utilizado y de los datos recopilados, pero se requerirá cierta cantidad de manipulación de datos para cualquier aplicación de aprendizaje automático.

##### Codificación One-Hot

El primer paso para nosotros se conoce como codificación one-hotde los datos Este proceso toma variables categóricas, como Diabetes, y las convierte en una representación numérica sin un orden arbitrario

In [ ]:
# One Hot Encoding
dataset = pd.get_dummies(dataset)

dataset.head(5)

# Lista de características para uso posterior
features = dataset.drop('NIHSS_INICO_ACV', axis = 1)
feature_list = list(features.columns)

# 5-. Entrenar el modelo en los datos de entrenamiento

El rf_exp usa la misma cantidad de árboles de decisión (n_estimadores) pero está entrenado en el conjunto de datos más largo con 3 características adicionales. El estado aleatorio es simplemente el número de lote del conjunto generado aleatoriamente en cualquier operación. Podemos especificar este número de lote siempre que queramos el mismo juego de nuevo. 

<h4>n_estimators : </h4>este es el número de árboles que desea construir antes de tomar la votación máxima o los promedios de las predicciones. Una mayor cantidad de árboles le brinda un mejor rendimiento pero hace que su código sea más lento

In [ ]:
# Dividimos los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

# X son nuestras variables independientes
X = dataset.drop('NIHSS_INICO_ACV', axis = 1)

# y es nuestra variable dependiente
y = dataset['NIHSS_INICO_ACV']

# Uso de Skicit-learn para dividir datos en conjuntos de entrenamiento y prueba 
# División 75% de datos para entrenamiento, 25% de datos para testb
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

## Creamos el modelo

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Creamos el modelo de Arbol de Decisión (y configuramos el número máximo de nodos-hoja)
rf = RandomForestRegressor(
            n_estimators = 10,
            criterion    = 'mse',
            max_depth    = None,
            max_features = 'auto',
            oob_score    = False,
            n_jobs       = -1,
            random_state = 0)
rf.fit(X_train, y_train)

# 6-. Predicciones sobre los datos de prueba
Se evalúa la capacidad predictiva del árbol inicial calculando el accuracy en el conjunto de test.

In [ ]:
# Predicción Entrenamiento 
prediccionEntreno = dtc.predict(X_train)

# Predicción Tests
prediccionTests = dtc.predict(X_test)

## Fase de Entrenamiento

In [ ]:
from sklearn import metrics

print("Entrenamiento - Accuracy :", metrics.accuracy_score(y_train, prediccionEntreno))

### Matriz de Confusión

En el campo de la inteligencia artificial y en especial en el problema de la clasificación estadística, una matriz de confusión es una herramienta que permite la visualización del desempeño de un algoritmo que se emplea en aprendizaje supervisado.

In [ ]:
from matplotlib import pyplot as plot
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

matriz = confusion_matrix(y_train, prediccionEntreno)

plot_confusion_matrix(conf_mat=matriz, figsize=(6,6), show_normed=False)
plot.tight_layout()

In [ ]:
print("Entrenamiento - Reporte de clasificación:\n", metrics.classification_report(y_train, prediccionEntreno))

# 7-. Comparar las predicciones con los objetivos conocidos del conjunto de pruebas y calcule las métricas de rendimiento
## Fase de validación  (Usando datos de Test)

In [ ]:
print("Promedio - Accuracy :", metrics.accuracy_score(y_test, prediccionTests))

### Matriz de Confusión

In [ ]:
matriz = confusion_matrix(y_test, prediccionTests)

plot_confusion_matrix(conf_mat=matriz, figsize=(6,6), show_normed=False)
plt.tight_layout()

In [ ]:
print("Entrenamiento - Reporte de clasificación:\n", metrics.classification_report(y_test, prediccionTests))